In [ ]:
###################################################################################################
#
#  Welcome to the fourth MAST-ML tutorial notebook, MASTML_Tutorial_4_Models_and_Tests.ipynb! 
#  In this notebook, we will learn how to run a few different types of models on a select
#  dataset, and conduct a few different types of data splits to evaluate our model performance. In
#  this tutorial, we will:
#       1. Run a variety of model types from the scikit-learn package
#       2. Run a bootstrapped ensemble of linear ridge regression models
#       3. Compare performance of scikit-learn's gradient boosting method and XGBoost
#       4. Compare performance of scikit-learn's neural network and Keras-based neural network regressor
#       5. Compare model performance using random k-fold cross validation and leave out group
#          cross validation
#       6. Explore the limits of model performance when up to 90% of data is left out using
#          leave out percent cross validation
#
###################################################################################################

In [ ]:
#####################################
#
# Task 0: Setting up MAST-ML in Colab
#
#####################################

In [ ]:
# If you are working on Google Colab and need to install MAST-ML, 
# begin by cloning the relevant branch of MAST-ML to the Colab session
# and install the needed dependencies:

!git clone --single-branch --branch dev_Ryan_2020-12-21 https://github.com/uw-cmg/MAST-ML
!pip install -r MAST-ML/requirements.txt

In [ ]:
# Sync your Google drive to Colab so that we can save MAST-ML results to our Google
# Drive. If we save to the Colab session, the data will be deleted when the session 
# ends.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# We need to add the MAST-ML folder to our sys path so that python can find the modules
import sys
sys.path.append('MAST-ML')

In [1]:
# Here we import the MAST-ML modules used in this tutorial
from mastml.mastml import Mastml
from mastml.datasets import LocalDatasets
from mastml.models import SklearnModel, EnsembleModel
from mastml.preprocessing import SklearnPreprocessor
from mastml.data_splitters import SklearnDataSplitter, NoSplit, LeaveOutPercent

Using TensorFlow backend.


In [2]:
# Set the name of the savepath to save MAST-ML results to
SAVEPATH = 'drive/MyDrive/MASTML_tutorial_4_Models_and_Tests'

# Initialize the MAST-ML run, write savepath
mastml = Mastml(savepath=SAVEPATH)
savepath = mastml.get_savepath

# When the above command is run, a new folder with the name designated SAVEPATH is created.
# This is where all of the output for the current MAST-ML run will be saved to.
# Note that you can perform multiple runs with the same folder name, and the current datetime
# will be appended to the name so that no data is lost or overwritten.

drive/MyDrive/MASTML_results_GettingStarted_1 not empty. Renaming...


In [ ]:
##########################################################################
#
# Task 1: Run a variety of model types from the scikit-learn package
#
##########################################################################

In [ ]:
# This tutorial is a work in progress- check back soon for updates!

In [ ]:
##########################################################################
#
# Task 2: Run a bootstrapped ensemble of linear ridge regression models
#
##########################################################################

In [ ]:
# This tutorial is a work in progress- check back soon for updates!

In [ ]:
############################################################################################
#
# Task 3: Compare performance of scikit-learn's gradient boosting method and XGBoost
#
############################################################################################

In [ ]:
# This tutorial is a work in progress- check back soon for updates!

In [ ]:
####################################################################################################################
#
# Task 4: Compare performance of scikit-learn's neural network and Keras-based neural network regressor
#
####################################################################################################################

In [ ]:
# This tutorial is a work in progress- check back soon for updates!

In [ ]:
####################################################################################################################
#
# Task 5: Compare model performance using random k-fold cross validation and leave out group cross validation
#
####################################################################################################################

In [ ]:
# This tutorial is a work in progress- check back soon for updates!

In [ ]:
####################################################################################################################
#
# Task 6: Explore the limits of model performance when up to 90% of data is left out using
#         leave out percent cross validation
#
####################################################################################################################

In [ ]:
# This tutorial is a work in progress- check back soon for updates!

In [ ]:
# You've now completed your fourth MAST-ML tutorial notebook! Now that you're more familiar with working
# with different types of models and cross validation tests to evaluate those models, we are ready to 
# move on to some more advanced fitting methods, like nested cross validation and model optimization
#
# The next example in this notebook series is titled MASTML_Tutorial_5_NestedCV_and_OptimizedModels.ipynb, 
# and will guide you through the process of assessing model performance on true test data, and generating the
# best optimized model for a specified cross validation test